# 3 - Apache Spark ML- Create machine learning models

In this chapter, you will:

• Build your first ML model with Spark ML

• Learn more Spark functionality and how to use it

In [1]:
from pyspark.sql import SparkSession 


spark = SparkSession.builder \
    .master('local[*]') \
    .appName("ApacheSparkML") \
    .getOrCreate()

df_train = spark.read.parquet("../../../data/detecting-twitter-bot-data/classified_train_data")
df_test = spark.read.parquet("../../../data/detecting-twitter-bot-data/classified_test_data")



✅ Task :

### Frequent Patterns Growth algorithm - FPGrowth()

Your data is clean and organized. It's your turn to create your **first** ML model with Spark.

Run the next code; it runs the **Frequent Patterns Growth** algorithm to extract patterns if those exist.

Tweak the minSupport and minCondifence.


<details><summary>Hint</summary>
<p>

Change the minSupport and minCondifence to 0.1 and see what happens.

</p>
</details>



In [4]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="description", minSupport=0.21, minConfidence=0.1)
fpGrowth_model = fpGrowth.fit(df_train)

# Display frequent itemsets.
fpGrowth_model.freqItemsets.show()

# Display generated association rules.
fpGrowth_model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
fpGrowth_model.transform(df_train).toPandas().transpose()


+-----+----+
|items|freq|
+-----+----+
|[and]| 397|
+-----+----+

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+



,0,1,2,3,4,5,6,7,8,9,...,1679,1680,1681,1682,1683,1684,1685,1686,1687,1688
screen_name,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
location,1,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
description,[25],"[, by, @mcmoots, your, Swifties, Follow, tweet...","[, follow, _Ùªâ, @jalfiebutler, for, back, Fol...","[, gene, elements, papers, #PubMed#bioRxiv, to...","[, here, @thricedotted, an, there, image, or]","[, my, repeats, to, not, every, @-reply, minut...","[, starting, @avoision, Twitter, posting, with...","[@LethargicJOE, this, get, your, follow, You, ...","[@jeffthompson_, guitar, by, Twitter, generati...","[ADVICE, B‰ÐÓOT, SO‰Ð_URCE, NOT, GOO‹ÎÇD, FOR,...",...,"[at, hats, Inc, Galois, of, many, Wearer]","[by, fintech, for, @Aden_76, little, twitter, ...","[elevate, 2, my, to, married, about, Proud, of...","[girl, broke, I, guarantee, a, heart, to, is, ...",[http:tcoYfHoThSDVQ],"[https:tcojgcn7jPp0t, scores, Hourly, @deer_fu...","[it, TWITTER, SHAWNNA, Booking:, Getting, to, ...","[link, #SpeakResponsibly, Host, @FoxSoccer, TV...","[mocinha, o, í©, de, mas, meio, ogra, coraí_í£o]","[senior, watercooler, @Mashable, reporter]"
followers_count,NaN,110.0,34.0,9.0,314.0,64.0,136.0,189.0,119.0,50.0,...,0.0,7230.0,5.0,132.0,89.0,598.0,122108.0,458212.0,646.0,2932.0
friends_count,1.0,111.0,787.0,0.0,0.0,0.0,1.0,366.0,1.0,2.0,...,247.0,947.0,662.0,788.0,329.0,1.0,5790.0,710.0,446.0,997.0
listed_count,NaN,14.0,0.0,7.0,50.0,13.0,21.0,0.0,21.0,10.0,...,0.0,1019.0,1.0,0.0,2.0,42.0,923.0,4944.0,2.0,107.0
favourites_count,NaN,0.0,7.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,...,28.0,162.0,169.0,0.0,3.0,0.0,223.0,4586.0,13440.0,1400.0
verified,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,1
statuses_count,NaN,3609.0,2.0,130.0,2978.0,180.0,34446.0,36.0,18719.0,1228.0,...,24.0,75122.0,208.0,355.0,498.0,9684.0,24891.0,7554.0,25292.0,619.0
status,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


What did you get?



<details><summary>Check me after running the previous code multipale times</summary>
<p>

When tweaking the minSupport=0.21 and minConfidence=0.1

You get:

|items|freq|
|----|:----:|
|[and]| 389|


</p>
</details>


---

✅ **Task :**

### LinearRegression functionality

It's your turn to create your **second** ML model with Spark - Linear Regression.




<details><summary>What is linear regression used for?</summary>
<p>
Linear regression is a common Statistical Data Analysis technique. It is used to determine the extent to which there is a linear relationship between a dependent variable and one or more independent variables.
</p>
</details>


**But**, before jumping right into it, you should know:

Spark ML Linear Regression **input** is:
1. `label` of type Double - our classification
2. `features` of type - `Vector[Double]` - Vector of Double, turn all columns into one column named features.
Hence, you will transform all _numeric_ columns into one Vector.

Leave the `description` out as it is not relevant for your next task.
    
For creating `features` column we use Vector Assembler
``` python
from pyspark.ml.feature import VectorAssembler
vecAssembler = VectorAssembler(inputCols=["a", "b", "c"], outputCol="features",handleInvalid = "skip")
new_df = vecAssembler.transform(dataFrame)
new_df.show()

```
<details><summary><b>Did you know!</b></summary>
<p>

Vector has two types in Spark:
    
    1. Dense Vector
    2. Sparse Vector
    
_Sparse vector_ is when you have many values in the vector as zero or null.

_Dense vector_ is when most of the values in the vector are non zero or non-null.

</p>
</details>


🤔 **Question**

Have you noticed `handleInvalid` param?  
```python
handleInvalid = "skip"
```




What happens if you remove it?

Validate yourself with `vecAssembler.show()`

Notice! You have a new DataFrame now. 

Remember to check yourself and work with the new DataFrame

In [5]:
# your code goes here
from pyspark.ml.feature import VectorAssembler
vecAssembler = VectorAssembler(inputCols=["a", "b", "c"], outputCol="features",handleInvalid = "skip")
new_df = vecAssembler.transform(dataFrame)
new_df.show()

NameError: name 'dataFrame' is not defined

---

✅ **Task :** 

To run ML training phase on the scalar vector you need to create DataFrame out of it.

`bot` and `features` are the **only** columns that we care about.

Do it with drop function:
    
```python

output = df_train.drop("val1","val2")
```




In [2]:
# run this:
output_train = df_train.drop('screen_name','location','followers_count','friends_count','listed_count','favourites_count','verified','statuses_count','status','default_profile','name')
output_train.show()

After turning _numeric_ columns into one `features` column and dropping `description`:

We got left with creating `label` column:

Create a new DataFrame with `label` column:
Code sample:
```python
df_for_lr  = output_train.selectExpr("features", "bot as label")
df_for_lr.show()

df_for_lr.toPandas().transpose()
```


Notice that now `df_for_lr` is your new DataFrame for creating `LinearRegression` model

In [3]:
# your code goes here

Run the next code.
Check out where you use the new DataFrame - `df_for_lr`

It creates a machine learning model out of linear regression.

Tweak the `maxIter`,`regParam` and `elasticNetParam` !

In [ ]:
from pyspark.ml.regression import LinearRegression


lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(df_for_lr)

# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

What did you get?

How does it look like?

What is `r2`? `r2` is a shortcut for R Square: 
>R-squared is a statistical measure of how close the data are to the fitted regression line. It is also known as the coefficient of determination, or the coefficient of multiple determination for multiple regression.

What is RMSE?
> Root Mean Square Error (RMSE) is the standard deviation of the residuals (prediction errors). Residuals are a measure of how far from the regression line data points are; RMSE is a measure of how spread out these residuals are. In other words, it tells you how concentrated the data is around the line of best fit.

Try to play with the parameters and watch how they change.

RMSE alone is meaningless until we compare with the actual `label` value, such as mean, min and max. 
After such comparison, our RMSE looks pretty good.

Compare `RMSE` and `mean` output.
After such comparison, our RMSE looks pretty good.


In [ ]:
df_for_lr.describe().show()

You built 2 machine learning models!

However, didn't get the best results.

It's OK!

And absolutly normal.

In chapter 4 you learn how to improve it.

### To load the models later, Save them to file:

In [ ]:
lrModel.save("linearRegression_model")

In [ ]:
fpGrowth_model.save("fpGrowth_model")

## Well Done! 👏👏👏
## You just finished: Apache Spark ML and created machine learning models¶
## Next Chapter: Evaluating ML models and using pipelines 